In [104]:
from hanziconv import HanziConv
line='你好，这只是个测试'
print(HanziConv.toTraditional(line))

你好，這隻是個測試


In [47]:
#這個方法可以彈藥另外存成記事本且要編碼維UTF8
from hanziconv import HanziConv
with open('data/spam/'+'5', 'r',encoding = 'gbk') as f:
            # 过滤掉非中文字符
            for line in f.readlines():
                pattern = re.compile('[^\u4e00-\u9fa5]')
                line = pattern.sub("", line)
                data=line.strip()
                print(HanziConv.toTraditional(line))
                # 将邮件中出现的词保存到wordsList中
               


























當當專業圖書網提供央視焦點訪談節目嚮全國觀眾大力推薦的好書話說中國全套共捲光盤
話說中國叢書係民族精神史詩重點齣版工程之一該工程已被列為年中央宣傳部項重點工程之一這套書用餘則故事位曆史人物幅以上圖片條曆史文化百科知識串起瞭泱泱中華年的曆史尤其是彆具匠心地以故事為經以圖片為緯的編排形式讓普通大眾乃至青少年讀者在閱讀一個個故事觀賞一幅幅圖畫的過程中輕鬆愉快地認知瞭中華五韆年的燦爛文明
更多專業圖書敬請登陸當當專業圖書網本郵件采用國際列車發送可能是國內最先進的郵件發送軟件獨傢技術您想獲得網上最新發布的供求信息情報嗎您想獲取成韆上萬個網站上的最新商機嗎您想獲得企業最新公布的産品信息嗎您想獲知競爭對手在網絡上的營銷情況嗎您想獲得行業的最新信息嗎
或許您已經給網絡上的浩瀚信息所淹沒您花費瞭大量的時間與精力卻所獲甚少
供求情報通捕捉最新商機尋找最新客戶一旦擁有成功在手詳情請參考網站免費索取試用



In [53]:
#!usr/bin/env python  
# -*- coding:utf-8 -*-

""" 
@author:yzk13 
@time: 2018/03/25 
"""
import jieba
import os

class Classifier:
    """
    分类器
    """
    def getStopWords(self):
        """
        获得停词列表
        :return stopList 停词列表
        """
        stopList = []
        with open('data/stopWords.txt', 'r',encoding='gbk') as f:
            for line in f.readlines():
                # 去除末尾\n
                stopList.append(line[:-1])
        return stopList

    def getFileList(self, filePath):
        """
        获取文件名称列表
        :param: filePath 文件夹路径
        :return fileNameList 文件名称列表
        """
        fileNameList = os.listdir(filePath)
        return fileNameList

    def getWordsList(self, content, wordsList, stopList):
        """
        获取词表词典, 由于要对多文本进行统计，所以这里不返回，一直在后面添加
        :param content: 文本内容
        :param wordsList: 词表
        :param stopList: 停词表
        """
        resultList = list(jieba.cut(content))
        for result in resultList:
            # 停词
            if result not in stopList and result.strip() != '' and result != None:
                if result not in wordsList:
                    wordsList.append(result)

    def listToDict(self, wordsList, wordsDict):
        """
        list转dict
        :param wordsList: 词列表
        :param wordsDict: 词字典
        """
        for item in wordsList:
            if item in wordsDict.keys():
                wordsDict[item] += 1
            else:
                wordsDict[item] = 1

    def getProbWord(self, testDict, normalDict, spamDict, numNormal, numSpam):
        """
        计算对分类结果影响最大的15个词
        :param testDict: 测试数据字典
        :param normalDict: 正常邮件字典
        :param spamDict: 垃圾邮件字典
        :param numNormal: 正常邮件的数量
        :param numSpam: 垃圾邮件的数量
        :return wordProbList: 对分类结果影响最大的15个词
        """
        wordProbList = {}
        for word, num in testDict.items():
            # 当词不在垃圾邮件词表中，在正常邮件词表中，计算概率
            if word in spamDict.keys() and word in normalDict.keys():
                # 求类先验概率
                # 正常邮件
                pw_n = normalDict[word] / numNormal
                # 垃圾邮件
                pw_s = spamDict[word] / numSpam
                ps_w = pw_s / (pw_s + pw_n)
                wordProbList[word] = ps_w
            # 当词在垃圾邮件词表中，不在正常邮件词表中，计算概率
            if word in spamDict.keys() and word not in normalDict.keys():
                pw_s = spamDict[word] / numSpam
                pw_n = 0.01
                ps_w = pw_s / (pw_s + pw_n)
                wordProbList[word] = ps_w
            # 当词在垃圾邮件词表中，而且在正常邮件词表中，计算概率
            if word not in spamDict.keys() and word in normalDict.keys():
                pw_s = 0.01
                pw_n = normalDict[word] / numNormal
                ps_w = pw_s / (pw_s + pw_n)
                wordProbList[word] = ps_w
            # 当词不在垃圾邮件词表中，也不在正常邮件词表中，计算概率
            if word not in spamDict.keys() and word not in normalDict.keys():
                wordProbList[word] = 0.5  # 0.4
        sorted(wordProbList.items(), key=lambda d: d[1], reverse=True)[0:15]
        return wordProbList

    def calBayes(self, wordList, spamDict, normalDict):
        """
        计算贝叶斯概率
        :param wordList: 词表
        :param spamDict: 垃圾邮件词语字典
        :param normalDict: 正常邮件词语字典
        :return: 概率
        """
        ps_w = 1
        ps_n = 1

        with open('wordsProb.txt', 'a', encoding='gbk') as f:
            for word, prob in wordList.items():
                f.write(word + ":" + str(prob) + "\n")
                ps_w *= prob
                ps_n *= 1 - prob
            p = ps_w / (ps_w + ps_n)
        return p

    def calAccuracy(self, testResult):
        """
        计算精度
        :return:
        """
        rightCount = 0
        errorCount = 0
        for name, catagory in testResult.items():
            if (int(name) < 1000 and catagory == 0) or (int(name) > 1000 and catagory == 1):
                rightCount += 1
            else:
                errorCount += 1
        return rightCount / (rightCount + errorCount)


In [54]:
import re
if __name__ == '__main__':
    classifier = Classifier()

    # 定义变量
    # 词频字典
    normalDict = {}
    spamDict = {}
    testDict = {}

    # 保存每封邮件中出现的词
    wordsList = []
    wordsDict = {}

    # 保存结果
    testResult = {}

    # 获取文件夹文件文件名列表
    listNormal = classifier.getFileList(r'data\normal')
    listSpam = classifier.getFileList(r'data\spam')
    # 测试集中文件名低于1000的为正常邮件
    listTest = classifier.getFileList(r'data\test')

    # 正常邮件与垃圾邮件的数量
    numNormal = len(listNormal)
    numSpam = len(listSpam)

    # 获得停词表
    stopList = classifier.getStopWords()

    # 获取正常邮件中的词频
    for fileName in listNormal:
        # 清空词表
        wordsList.clear()
        with open('data/normal/'+fileName, 'r',encoding='gbk') as f:
            # 过滤掉非中文字符
            for line in f.readlines():
                pattern = re.compile('[^\u4e00-\u9fa5]')
                line = pattern.sub("", line)
                # 将邮件中出现的词保存到wordsList中
                classifier.getWordsList(line, wordsList, stopList)
            # 统计每个词在所有邮件中出现的次数
            classifier.listToDict(wordsList, wordsDict)
    normalDict = wordsDict.copy()

    wordsDict.clear()
    # 获取垃圾邮件中的词频
    for fileName in listSpam:
        # 清空词表
        wordsList.clear()
        with open('data/spam/'+fileName, 'r',encoding='gbk') as f:
            # 过滤掉非中文字符
            for line in f.readlines():
                pattern = re.compile('[^\u4e00-\u9fa5]')
                line = pattern.sub("", line)
                # 将邮件中出现的词保存到wordsList中
                classifier.getWordsList(line, wordsList, stopList)
            # 统计每个词在所有邮件中出现的次数
            classifier.listToDict(wordsList, wordsDict)
    spamDict = wordsDict.copy()

    # 将结果写入文件wordsProb中,判断是否存在，存在则删除，因为后面执行追加写入
    if os.path.exists('wordsProb.txt'):
        os.remove('wordsProb.txt')

    # 测试邮件
    for fileName in listTest:
        testDict.clear()
        wordsDict.clear()
        wordsList.clear()
        with open('data/test/' + fileName, 'r' ,encoding='gbk') as f:
            # 过滤掉非中文字符
            for line in f.readlines():
                pattern = re.compile('[^\u4e00-\u9fa5]')
                line = pattern.sub("", line)
                # 将邮件中出现的词保存到wordsList中
                classifier.getWordsList(line, wordsList, stopList)
            # 统计每个词在所有邮件中出现的次数
            classifier.listToDict(wordsList, wordsDict)
            testDict = wordsDict.copy()

        # 计算每个文件中对分类影响最大的15个词
        wordProbList = classifier.getProbWord(testDict, normalDict, spamDict, numNormal, numSpam)

        # 对每封邮件得到的15个影响率最大的词计算贝叶斯概率
        p = classifier.calBayes(wordProbList, spamDict, normalDict)
        if (p > 0.5):
            testResult[fileName] = 1
        else:
            testResult[fileName] = 0
    testAccuracy = classifier.calAccuracy(testResult)
    # 测试邮件分类结果
    with open('result.txt', 'w' ,encoding='gbk') as f:
        for i, ic in testResult.items():
            f.write(str(i) + "," + str(ic) + "\n")

    print('准确率：' + str(testAccuracy))

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\zxc98\AppData\Local\Temp\jieba.cache
Loading model cost 0.709 seconds.
Prefix dict has been built succesfully.


准确率：0.9387755102040817


In [ ]:
import codecs
import pandas as pd
files = ["1.txt","2.txt"]
list_ = ""
all_text = []
for file in files:         
    with codecs.open(file, "r", "utf-8") as f:
        text=[]
        for line in f:
            content = line.strip().split('\t')
            # 对每一行的内容进行处理
            text = text+content
        text = " ".join(text)
    all_text.append(text)
data = pd.DataFrame(all_text,columns=['text'])
data
#         list_.append(pd.DataFrame(all_text,columns=['text']))  # Series对象也可
# raw_data = pd.concat(all_text)
# print(raw_data)